# Domain-Driven Design (DDD)

## Introduction

**Domain-Driven Design (DDD)** is an approach to software development that emphasizes the importance of the business domain and domain logic. The primary goal of DDD is to create a shared understanding of the domain between developers and domain experts and to translate that understanding into a software model.


## Key Concepts and Principles

1. **Domain**: The sphere of knowledge and activity around which the business logic revolves. For example, in an e-commerce application, the domain would include products, orders, customers, etc.

2. **Ubiquitous Language**: A common language used by both developers and domain experts to describe the domain. This language should be used in code, documentation, and discussions.

3. **Bounded Context**: A boundary within which a particular domain model is defined and applicable. Different parts of a large system might have different models and bounded contexts.

4. **Entities**: Objects that have a distinct identity that runs through time and different states. For example, a customer or an order in an e-commerce system.

5. **Value Objects**: Objects that describe some characteristics or attributes but have no distinct identity. For example, an address or a date range.

6. **Aggregates**: A cluster of domain objects that can be treated as a single unit. An aggregate always has a root entity, referred to as the aggregate root, that guarantees the consistency of changes within the aggregate.

7. **Repositories**: Mechanisms for encapsulating storage, retrieval, and search behavior which emulates a collection of objects.

8. **Factories**: Encapsulate the creation of complex objects and aggregates.

9. **Services**: Operations or functionalities that don’t naturally fit within an entity or value object.

## Step-by-Step Guide with an e-commerce Application

### Step 1: Define the Domain and Ubiquitous Language

Let's consider an example of an e-commerce application. We'll start by defining the domain and creating a ubiquitous language.

- **Domain**: E-commerce
- **Entities**: Product, Customer, Order
- **Value Objects**: Address, Money
- **Aggregates**: Order (comprising OrderLine, Customer, ShippingAddress)
- **Services**: OrderService, PaymentService
- **Repositories**: ProductRepository, OrderRepository

### Step 2: Define Entities and Value Objects

We'll create a few classes to represent entities and value objects.

In [22]:
from dataclasses import dataclass
from typing import List

# Value Object
@dataclass(frozen=True)
class Money:
    amount: float
    currency: str

# Value Object
@dataclass(frozen=True)
class Address:
    street: str
    city: str
    postal_code: str
    country: str

# Entity
@dataclass
class Customer:
    customer_id: int
    name: str
    email: str
    shipping_address: Address

# Entity
@dataclass
class Product:
    product_id: int
    name: str
    price: Money

# Entity
@dataclass
class OrderLine:
    product: Product
    quantity: int
    price: Money

# Aggregate Root
@dataclass
class Order:
    order_id: int
    customer: Customer
    shipping_address: Address
    order_lines: List[OrderLine]
    total: Money

### Step 3: Define Repositories

Repositories are responsible for retrieving and storing aggregates. We'll create interfaces for repositories.

In [23]:
from abc import ABC, abstractmethod

class OrderRepository(ABC):
    @abstractmethod
    def save(self, order: Order) -> None:
        pass
    
    @abstractmethod
    def find_by_id(self, order_id: int) -> Order:
        pass

class ProductRepository(ABC):
    @abstractmethod
    def find_by_id(self, product_id: int) -> Product:
        pass

### Step 4: Define Services

Services perform operations that don't naturally fit within an entity or value object. We'll define an `OrderService`.

In [24]:
class OrderService:
    def __init__(self, order_repository: OrderRepository, product_repository: ProductRepository):
        self.order_repository = order_repository
        self.product_repository = product_repository

    def create_order(self, customer: Customer, shipping_address: Address, product_quantities: List[tuple]) -> Order:
        order_lines = []
        total_amount = 0.0
        currency = 'EUR'  # Assuming a single currency for simplicity

        for product_id, quantity in product_quantities:
            product = self.product_repository.find_by_id(product_id)
            order_line = OrderLine(
                product=product,
                quantity=quantity,
                price=Money(amount=product.price.amount * quantity, currency=product.price.currency)
            )
            order_lines.append(order_line)
            total_amount += order_line.price.amount

        order = Order(
            order_id=123,  # Ideally, this should be generated uniquely
            customer=customer,
            shipping_address=shipping_address,
            order_lines=order_lines,
            total=Money(amount=total_amount, currency=currency)
        )

        self.order_repository.save(order)
        return order

### Step 5: Implement Repositories (Mock Implementation)

To test our service, we'll create mock implementations of our repositories.

In [25]:
class InMemoryOrderRepository(OrderRepository):
    def __init__(self):
        self.orders = {}

    def save(self, order: Order) -> None:
        self.orders[order.order_id] = order
    
    def find_by_id(self, order_id: int) -> Order:
        return self.orders.get(order_id)

class InMemoryProductRepository(ProductRepository):
    def __init__(self):
        self.products = {
            1: Product(1, "Laptop", Money(1000.0, "EUR")),
            2: Product(2, "Mouse", Money(50.0, "EUR"))
        }

    def find_by_id(self, product_id: int) -> Product:
        return self.products.get(product_id)

### Step 6: Use the Service to Create an Order

Now, let's use our `OrderService` to create an order.

In [26]:
# Setup repositories
order_repository = InMemoryOrderRepository()
product_repository = InMemoryProductRepository()

# Create service
order_service = OrderService(order_repository, product_repository)

# Create a customer and address
customer = Customer(customer_id=1, name="Albert Einstein", email="einstein@sample.com",
                    shipping_address=Address(street="123 Main St", city="Munich", postal_code="80888", country="Germany"))

# Create an order
order = order_service.create_order(customer, customer.shipping_address, [(1, 1), (2, 2)])

print(order)

Order(order_id=123, customer=Customer(customer_id=1, name='Albert Einstein', email='einstein@sample.com', shipping_address=Address(street='123 Main St', city='Munich', postal_code='80888', country='Germany')), shipping_address=Address(street='123 Main St', city='Munich', postal_code='80888', country='Germany'), order_lines=[OrderLine(product=Product(product_id=1, name='Laptop', price=Money(amount=1000.0, currency='EUR')), quantity=1, price=Money(amount=1000.0, currency='EUR')), OrderLine(product=Product(product_id=2, name='Mouse', price=Money(amount=50.0, currency='EUR')), quantity=2, price=Money(amount=100.0, currency='EUR'))], total=Money(amount=1100.0, currency='EUR'))


## Conclusion

By following DDD principles, you can create software that accurately reflects the business domain and is easier to maintain and extend over time.